In [2]:
import sys
sys.path.append('../../lib/')

In [3]:
from analysis import *
import json
import requests
import time
import shutil

In [3]:
tweets_with_images = {}

In [4]:
with open('../../key.json') as f:
	key = json.load(f)
	bearer_token = key['BEARER_TOKEN']

In [5]:
tweets_list_possable_bad = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/posable_bad_tweets.json"))
tweets_list_possable_good = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/all_good_tweets_v4.json"))
tweets_list_false_positives = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/false_postives_from_v3_ivory.json"))
tweets_list = tweets_list_possable_bad + tweets_list_possable_good + tweets_list_false_positives

In [6]:
print(len(tweets_list))

1350


In [7]:
# Get timeline
def get_tweet(tweet_id):
	# Create the url
	url = 'https://api.twitter.com/2/tweets/{}'.format(tweet_id) + '?expansions=attachments.media_keys&&media.fields=url'
	# Create request headers
	headers = {
		'authorization': 'Bearer ' + bearer_token,
		'user-agent': 'v2FilteredStreamPython',
	}
	# Make the request
	response = requests.get(url, headers=headers)
	return response.json()

In [8]:
tweets_with_images = {}
tweets_with_images["tweets"] = []
# Go through each tweet
for tweet in tweets_list:
    try:
        tweet_response = get_tweet(analysis.get_tweet_id(tweet))
        if tweet_response['data']:
            tweets_with_images["tweets"].append(tweet_response)
    except:
        print(analysis.get_tweet_text(tweet)) 
    time.sleep(1)

Baule figure with beautiful carving detail from Ivory Coast, Early 20th C. Measuring 40.5 cm
 
Provenance: Private collection, California, USA

For sale from @david_malik_african_art 
https://t.co/TGeHiOaDby https://t.co/bFysEUufw8
Such a wonderful vegan alternative to Ivory. No animals were harmed in the making of these earrings. X

#vegan #ivory #earlings #jewellery #sustainable https://t.co/qcKTUrChQV
RT @Wilson55Auction: SOLD £5,800 - Sarah Biffin (British 1784-1850) Portrait of a military officer, miniature, watercolour on ivory #auctio…
Specialists with the appropriate knowledge are able to judge whether an item was made in an earlier era”. BADA has therefore proposed “third-party verification for carvings or items comprising mainly ivory

https://t.co/efx0G5e6zn https://t.co/kN2BARaE7V
Our fine art restorer can be seen meticulously reattaching the arm of this ivory figurine.

#restoration #ceramics #sculpture #ivory #inprogress https://t.co/jcCRrEC9sJ
Coming to us with one leg m

KeyboardInterrupt: 

In [9]:
# Write to file
with open('../../data/dataset/tweets_with_images.json', 'w') as f:
	json.dump(tweets_with_images, f)

In [5]:
json_image_path = '../../data/dataset/tweets_with_images.json'
# load json file
with open(json_image_path) as json_file:
	data = json.load(json_file)

In [6]:
for tweet in data["tweets"]:
    image_count = 0
    try:
        for image in tweet['includes']['media']:
            image_url = image['url']
            print(image_url)
            # Download image
            r = requests.get(image_url, stream=True)
            
            # Get image type from url
            image_type = image_url.split('.')[-1]
            if r.status_code == 200:
                with open('../../data/dataset/images/' + str(tweet['data']['id']) + "_" + str(image_count) + "." + image_type, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
            image_count += 1
    except:
        print("Error id: " + str(tweet['data']['id']))


https://pbs.twimg.com/media/EOfhQPyXkAI7um2.png
https://pbs.twimg.com/media/EOUbGtNWoAMuDNi.jpg
https://pbs.twimg.com/media/EN1R8i7XsAIAEED.png
https://pbs.twimg.com/media/ENXuDhRXkAUS2QJ.png
https://pbs.twimg.com/media/ENIudOBWkAAmGOE.jpg
https://pbs.twimg.com/media/EKcxagfWwAAahd4.png
https://pbs.twimg.com/media/EKSGJDXW4AIp7KZ.png
https://pbs.twimg.com/media/EKDinLDWwAA9wVB.png
https://pbs.twimg.com/media/EJ-Eo9oW4AY0Udt.png
https://pbs.twimg.com/media/EJzq48vWwAAgMhv.png
https://pbs.twimg.com/media/EJuorSOXsAAUvJf.png
https://pbs.twimg.com/media/EJpeuidWoAEqX_C.png
https://pbs.twimg.com/media/EJkriQJXkAEhqSG.png
https://pbs.twimg.com/media/EJgT5OTXsAEcwga.jpg
https://pbs.twimg.com/media/EJUvfbBXsAAGNZe.png
https://pbs.twimg.com/media/EJPndq-XUAUIuYZ.png
https://pbs.twimg.com/media/EJFPWrNXsAAVOZ0.png
https://pbs.twimg.com/media/EJA72oxXUAEFeHY.png
https://pbs.twimg.com/media/EI7XlPIW4AAzxyk.png
https://pbs.twimg.com/media/EI1sNSVWoAAB1IJ.png
https://pbs.twimg.com/media/EIr2qiOXYAA3